In [5]:
import hdf5_getters

In [17]:
h5 = hdf5_getters.open_h5_file_read("TRAAAAW128F429D538.h5")
duration = hdf5_getters.get_artist_terms_weight(h5)
print(duration)


[ 1.          0.89793596  0.88426185  0.84262975  0.84256301  0.83239282
  0.82577707  0.79859195  0.7431759   0.73850237  0.72505245  0.71389955
  0.67049417  0.65697231  0.65105613  0.65105612  0.65105597  0.65105592
  0.65105547  0.65105532  0.65105508  0.65105506  0.65105461  0.65105427
  0.65105376  0.65104997  0.6364043   0.63334971  0.61973455  0.61889383
  0.61419433  0.59579116  0.56220197  0.55067233  0.52897541  0.49021215
  0.38341077]
